###### **<span style="font-family: 'Palatino Linotype', serif;"> Ilum - School of Science (CNPEM) </span>**

## **<span style="font-family: 'Palatino Linotype', serif;"> Modeling Melodios </span>**

##### **<span style="font-family: 'Palatino Linotype', serif;"> Authors:</span>** <span style="font-family: 'Palatino Linotype', serif;"> Caio M. Leão Dantas, Enzo J. Xavier, Rafael A. S. Santos, Thomas W. Hannemann</span> 
##### **<span style="font-family: 'Palatino Linotype', serif;"> Advisor:</span>** <span style="font-family: 'Palatino Linotype', serif;"> Felipe David Crasto de Lima </span> 

___

## 1. Libraries and Setup
In this section, we import the necessary Python libraries for data manipulation (`numpy`, `pandas`) and visualization (`matplotlib`). We also include modules for file system handling (`os`, `glob`, `pathlib`), text processing (`re`) to parse musical notation, and auxiliary functions for math (`math`) and output formatting (`pprint`).

In [ ]:
import os
import glob 
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import re
import numpy as np
from math import pi
from pathlib import Path
from pprint import pprint

# Standard Sample Rate for audio generation
SR = 44_100 

## 2. Core Functions
Here we define the fundamental functions for the project:
1.  **Thermodynamic Functions**:
    * `calculate_energy`: Computes the "energy" of a melody based on the Ising-like model Hamiltonian ($H = - \sum J - \sum h$). Lower energy corresponds to a more statistically probable melody.
    * `metropolis_step`: Performs a single Metropolis-Hastings update. It attempts to change a note and accepts the change based on the Boltzmann probability distribution.
2.  **Audio Utility Functions**:
    * `render_melody` and `save_wav`: Synthesizers that convert the array of note strings into a listenable `.wav` file using simple sine wave synthesis.

In [ ]:
def inicializar_melodia_vetorial(L, escala_completa):
    """Initializes a random melody vector of length L."""
    return np.random.randint(0, len(escala_completa), size=L)

# -------------------------------------------------------------------------------

def calcular_energia_com_matriz(melodia, matriz_J, vetor_h):
    """
    Calculates the total energy (Hamiltonian) of the melody.
    E = - Sum(h_i) - Sum(J_{i, i+1})
    """
    energia = 0

    # 1. Adsorption energy (h): Intrinsic probability of each note
    for i in range(len(melodia)):
        energia -= vetor_h[melodia[i]]

    # 2. Interaction term (J): Transition probability between adjacent notes
    for i in range(len(melodia) - 1):
        nota_1 = melodia[i]
        nota_2 = melodia[i+1]
        energia -= matriz_J[nota_1, nota_2]
    return energia

# -------------------------------------------------------------------------------

def passo_metro_com_matriz(melodia, T, posicao_notas, matriz_J, vetor_h):
    """
    Executes one Monte Carlo step using the Metropolis algorithm.
    T: Temperature (controls randomness).
    """
    L = len(melodia)
    
    # Select a random position in the melody to mutate
    posicao = np.random.randint(0, L)
    nota_antiga = melodia[posicao]
    
    # Propose a new note different from the current one
    nova_nota = np.random.choice(posicao_notas)
    while nova_nota == nota_antiga:
        nova_nota = np.random.choice(posicao_notas)

    # Calculate the change in energy (dE) if the swap happens
    dE = 0

    # Interaction with left neighbor
    if posicao > 0:
        vizinho_esq = melodia[posicao-1]
        dE -= (matriz_J[vizinho_esq, nova_nota] - matriz_J[vizinho_esq, nota_antiga])
    
    # Interaction with right neighbor
    if posicao < L - 1:
        vizinho_dir = melodia[posicao+1]
        dE -= (matriz_J[nova_nota, vizinho_dir] - matriz_J[nota_antiga, vizinho_dir])

    # Adsorption energy contribution (h)
    dE -= (vetor_h[nova_nota] - vetor_h[nota_antiga])

    # Metropolis Criterion:
    # If energy decreases (dE < 0), accept.
    # If energy increases, accept with probability exp(-dE/T).
    if dE < 0 or np.random.rand() < np.exp(-dE / T):
        melodia[posicao] = nova_nota  # Accept the change

    return melodia

# -------------------------------------------------------------------------------
# AUDIO GENERATION UTILITIES
# -------------------------------------------------------------------------------

def note_to_freq(note: str, a4: float = 440.0):
    """Converts a note string (e.g., 'C#4') to frequency in Hz."""
    if note is None:
        raise ValueError("Received None note.")
    note = note.strip()
    if note.upper() in {"REST", "_"}:
        return None
    
    # Regex to parse Note + Accidental + Octave
    m = re.fullmatch(r"([A-Ga-g])([#b]?)(-?\d+)", note)
    if not m:
        raise ValueError(f"Invalid note: {note!r}")
    
    letter, acc, octave = m.groups()
    letter = letter.upper()
    octave = int(octave)
    
    # Semitone offsets from C
    base = {"C":0, "D":2, "E":4, "F":5, "G":7, "A":9, "B":11}[letter]
    if acc == "#":
        base += 1
    elif acc == "b":
        base -= 1

    # MIDI note calculation (69 = A4)
    midi = (octave + 1) * 12 + base           
    return a4 * (2.0 ** ((midi - 69) / 12.0)) 

def adsr_envelope(n_samples: int, sr: int, attack=0.02, decay=0.05, sustain=0.85, release=0.05):
    """Generates a simple ADSR volume envelope."""
    a = int(attack * sr)
    d = int(decay * sr)
    r = int(release * sr)
    s = max(0, n_samples - (a + d + r))

    env = np.zeros(n_samples, dtype=np.float32)
    
    if a > 0:
        env[:a] = np.linspace(0.0, 1.0, a, endpoint=False, dtype=np.float32)
    idx = a
    if d > 0 and idx + d <= n_samples:
        env[idx:idx+d] = np.linspace(1.0, sustain, d, endpoint=False, dtype=np.float32)
    idx += d
    if s > 0 and idx + s <= n_samples:
        env[idx:idx+s] = sustain
    idx += s
    rem = n_samples - idx
    if rem > 0:
        env[idx:] = np.linspace(env[idx-1] if idx > 0 else sustain, 0.0, rem, dtype=np.float32)
    return env

def synth_note(freq: float, duration: float, sr: int = SR, harmonics=(1.0, 0.3, 0.1)):
    """Synthesizes a single note using additive synthesis (sine waves)."""
    n = int(duration * sr)
    t = np.arange(n, dtype=np.float32) / sr
    y = np.zeros_like(t)

    # Add harmonics
    for k, amp in enumerate(harmonics, start=1):
        y += amp * np.sin(2 * pi * k * freq * t)

    # Apply ADSR envelope
    y *= adsr_envelope(n, sr)
    return y.astype(np.float32)

def render_melody(notes, durations=None, per_note=0.5, gap=0.02, sr: int = SR):
    """Converts a list of note strings into a complete audio waveform."""
    if durations is None:
        durations = [per_note] * len(notes)
    if len(durations) != len(notes):
        raise ValueError("durations must match length of notes.")
    
    pieces = []
    gap_samples = np.zeros(int(gap * sr), dtype=np.float32) if gap > 0 else np.zeros(0, dtype=np.float32)

    for note, dur in zip(notes, durations):
        f = note_to_freq(note)
        if f is None:  # silêncio
            tone = np.zeros(int(dur * sr), dtype=np.float32)
        else:
            tone = synth_note(f, dur, sr)
        pieces.append(tone)
        if gap_samples.size:
            pieces.append(gap_samples)

    if not pieces:
        return np.zeros(0, dtype=np.float32)
    
    y = np.concatenate(pieces)
    # Normalize volume to avoid clipping
    peak = float(np.max(np.abs(y))) or 1.0
    y = (0.95 * y / peak).astype(np.float32)
    return y

def save_wav(path, y: np.ndarray, sr: int = SR):
    """Saves the waveform as a standard 16-bit WAV file."""
    import wave
    path = Path(path); path.parent.mkdir(parents=True, exist_ok=True)
    with wave.open(str(path), "wb") as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)  
        wf.setframerate(sr)
        data = (np.clip(y, -1.0, 1.0) * 32767.0).astype(np.int16).tobytes()
        wf.writeframes(data)

## 3. Data Loading and Preprocessing
In this step, we read the dataset of musical melodies (text files). 
* We define a standard **Musical Scale** (from C3 to C7).
* We implement a `normalizar_nota` function to handle enharmonics (e.g., converting flats 'Db' to sharps 'C#') to ensure consistency across the dataset.
* We filter the raw data, keeping only valid notes that exist within our defined scale.

In [ ]:
melodias_dados = []

# Dictionary to map flats to sharps (Enharmonic equivalents)
equivalencias = {
    'D-': 'C#',
    'E-': 'D#',
    'G-': 'F#',
    'A-': 'G#',
    'B-': 'A#'
}


def normalizar_nota(nota):
    """Standardizes note names (converts flats to sharps)."""
    if len(nota) == 2:  # e.g.: C4
        base, oitava = nota[0], nota[1]
    elif len(nota) == 3:  # e.g.: C#4 or D-4
        base, oitava = nota[:2], nota[2]
    elif len(nota) == 4:  # e.g.: A#10 (rare)
        base, oitava = nota[:2], nota[2:]
    else:
        return nota
    
    # Replace flat if found
    for flat, sharp in equivalencias.items():
        if base == flat:
            base = sharp
            break
    return base + oitava


# Define the complete allowable scale (C3 to C7)
escala_completa = [
    'C3', 'C#3', 'D3', 'D#3', 'E3', 'F3', 'F#3', 'G3', 'G#3', 'A3', 'A#3', 'B3',
    'C4', 'C#4', 'D4', 'D#4', 'E4', 'F4', 'F#4', 'G4', 'G#4', 'A4', 'A#4', 'B4',
    'C5', 'C#5', 'D5', 'D#5', 'E5', 'F5', 'F#5', 'G5', 'G#5', 'A5', 'A#5', 'B5',
    'C6', 'C#6', 'D6', 'D#6', 'E6', 'F6', 'F#6', 'G6', 'G#6', 'A6', 'A#6', 'B6',
    'C7'
]

# Load files
diretorio_dados = './dados/melodias/' # Adjust this path as needed
file_pattern = '*.txt' 
filepaths = glob.glob(os.path.join(diretorio_dados, file_pattern))


for filepath in filepaths:
    with open(filepath, 'r') as file:
        conteudo = file.read().split()
        melodia_normalizada = []
        
        for nota in conteudo:
            nota_norm = normalizar_nota(nota)
            if nota_norm in escala_completa:
                melodia_normalizada.append(nota_norm)
            else:
                # Optional: Print unknown notes for debugging
                # print(f"Unknown note: {nota} (normalized: {nota_norm})")
                pass   
        
        # Only add non-empty melodies
        melodias_dados.append(melodia_normalizada)

# Show first 3 processed melodies
print(melodias_dados[:3]) 


## 4. Training: Estimating Model Parameters (h and J)
We treat the musical corpus as a statistical system.
1.  **Index Mapping**: Map every string note to a numerical index (0 to 48).
2.  **Adsorption Energy ($h$)**: Derived from the frequency of **individual notes**. Highly frequent notes have higher $h$ (lower energy).
3.  **Interaction Matrix ($J$)**: Derived from the frequency of **note transitions** ($A \to B$). Frequent transitions have higher $J$ (lower energy).

In [ ]:
# Create mapping: Note string -> Integer Index
num_notas_escala = len(escala_completa)
mapa_notas_indice = {nota: i for i, nota in enumerate(escala_completa)}
posicao_notas = np.arange(num_notas_escala)

# ---------------- CALCULATION OF h (Note Probabilities) ----------------
todas_as_notas = [nota for melodia in melodias_dados for nota in melodia]
contagem_notas = {nota: todas_as_notas.count(nota) for nota in escala_completa}
total_notas_melodia = len(todas_as_notas)

# Normalize counts to get probabilities
probabilidade_notas = {nota: count / total_notas_melodia for nota, count in contagem_notas.items()}

# Create vector h and scale it (x1000) to define energy magnitude
vetor_h = np.array([probabilidade_notas[nota] for nota in escala_completa]) * 1000

# ---------------- CALCULATION OF J (Transition Matrix) ----------------
matriz_contagem_transicoes = np.zeros((num_notas_escala, num_notas_escala))
for melodia in melodias_dados:
    for i in range(len(melodia) - 1):
        nota_atual = melodia[i]
        proxima_nota = melodia[i+1]
        idx_atual = mapa_notas_indice[nota_atual]
        idx_proxima = mapa_notas_indice[proxima_nota]

        # Count transition A -> B
        matriz_contagem_transicoes[idx_atual, idx_proxima] += 1

# Normalize to probabilities
total_interacoes = np.sum(matriz_contagem_transicoes)
matriz_probabilidade_transicoes = matriz_contagem_transicoes / total_interacoes

# Create Matrix J and scale it
matriz_energia_J = matriz_probabilidade_transicoes * 1000

# Display J as a dataframe for inspection
matriz_energia_J_df = pd.DataFrame(matriz_energia_J, index=escala_completa, columns=escala_completa)

## 5. Visualization of Model Parameters
Here we visualize the learned statistics:
1.  **Note Probability Distribution ($h$)**: Shows which notes are most common (Tonal centers).
2.  **Transition Matrix ($J$)**: A heatmap showing which note transitions are preferred.

In [ ]:
# 1. Plot Note Probabilities
nomes_das_notas = list(probabilidade_notas.keys())
probabilidades = list(probabilidade_notas.values())

plt.figure(figsize=(18, 8))
plt.bar(nomes_das_notas, probabilidades, color='purple')
plt.title('Probability Distribution of Musical Notes in Dataset', fontsize=34)
plt.xlabel('Musical Notes', fontsize=24)
plt.ylabel('Probability', fontsize=24)
plt.xticks(rotation=90, fontsize=16)
plt.yticks(fontsize=16)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.savefig('./imagens/notes_probability.pdf', format='pdf')
plt.savefig('.imagens/notes_probability.png', format='png', dpi=300)
plt.show()

In [ ]:
# 2. Plot Transition Matrix Heatmap
plt.figure(figsize=(12, 10))
plt.title('Transition Count Matrix Between Musical Notes', fontsize=25)

# Display matrix
plt.imshow(matriz_contagem_transicoes, cmap='viridis', interpolation='none')
plt.gca().set_aspect('equal')

# Add grid lines
plt.gca().set_aspect('equal')  # <<< garante que cada célula seja um quadrado
plt.gca().set_xticks(np.arange(-0.5, matriz_contagem_transicoes.shape[1], 1), minor=True)
plt.gca().set_yticks(np.arange(-0.5, matriz_contagem_transicoes.shape[0], 1), minor=True)
plt.grid(which='minor', color='white', linestyle='-', linewidth=0.8)
plt.tick_params(which='minor', bottom=False, left=False)

# Axis labels
step = 4
plt.xticks(
    np.arange(0, len(escala_completa), step),
    escala_completa[::step],
    rotation=90,
    fontsize=16
)
plt.yticks(
    np.arange(0, len(escala_completa), step),
    escala_completa[::step],
    fontsize=16
)

plt.xlabel('Next Note', fontsize=20)
plt.ylabel('Current Note', fontsize=20)

cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=16)
cbar.set_label('Transition Count', fontsize=20)

plt.tight_layout()
plt.savefig('./imagens/transition_matrix.pdf', format='pdf')
plt.savefig('./imagens/transition_matrix.png', format='png', bbox_inches='tight', dpi=300)
plt.show()

# 6. Metropolis Monte Carlo Simulation (Generation)
We generate new melodies by simulating the system at different **Temperatures ($T$)**.
* **Low T**: System freezes into ground states (most probable notes/transitions).
* **High T**: System behaves randomly (noise).
* **Intermediate T**: Balance between structure and variation (Creativity).

For each temperature, we:
1.  Initialize a random melody.
2.  Run the MCMC algorithm for `50,000` steps.
3.  Plot the evolution of the melody and the energy minimization.
4.  Render the final melody to Audio (`.wav`).

In [ ]:
L_melodia = 16 + 1  # Length of melody
passos_simulacao = 50000  # Steps
temperaturas = [0.001, 10, 30, 1000] # Temperature sweep

num_melodias_finais = 100  # How many last states to visualize for "blur" effect

for temperatura in temperaturas:
    # 1. Random Initialization
    melodia_inicial = inicializar_melodia_vetorial(L_melodia, escala_completa)
    melodia_atual = np.copy(melodia_inicial)

    historico_energia = []
    ultimas_melodias = []  

    # 2. Simulation Loop
    for passo in range(passos_simulacao):
        melodia_atual = passo_metro_com_matriz(
            melodia_atual, temperatura, posicao_notas, matriz_energia_J, vetor_h
        )

        # Record energy every 100 steps
        if passo % 100 == 0:
            energia = calcular_energia_com_matriz(melodia_atual, matriz_energia_J, vetor_h)
            historico_energia.append(energia)
        
        # Store last N states for visualization
        if passo >= passos_simulacao - num_melodias_finais:
            ultimas_melodias.append(np.copy(melodia_atual))

    # Final processing
    melodia_final = melodia_atual
    melodia_inicial_notas = [escala_completa[i] for i in melodia_inicial]
    melodia_final_notas = [escala_completa[i] for i in melodia_final]

    print(f"--- Melody Generated at T={temperatura} ---")
    print("Initial:", melodia_inicial_notas)
    print("Final:  ", melodia_final_notas)
    print()

    # ---------------- PLOTTING ----------------    
    plt.figure(figsize=(20, 18))
    
    # Subplot 1: Initial Random State
    plt.subplot(2, 1, 1)
    plt.title(f"Initial Melody at T = {temperatura}", fontsize=30)
    plt.plot(melodia_inicial, marker='o', linestyle='-', color='purple')
    plt.xticks(fontsize=18)
    plt.yticks(
        range(len(escala_completa)),
        [escala_completa[i] if i % 12 == 0 else "" for i in range(len(escala_completa))],
        fontsize=18
    )
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.text(
        0.98, 0.02,
        f'Initial melody:\n{" ".join(melodia_inicial_notas)}',
        fontsize=20, ha='right', va='bottom',
        transform=plt.gca().transAxes,
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray', boxstyle='round,pad=0.3')
    )
    plt.xlabel('Position in Melody', fontsize=22)
    plt.ylabel('Musical Note', fontsize=22)
    # plt.legend(fontsize=17)

    # Subplot 2: Final State (with trace of last N steps)
    plt.subplot(2, 1, 2)
    plt.title(f'Final Melodies (last {num_melodias_finais}) at T = {temperatura}', fontsize=30)

    # Draw ghost traces 
    for mel in ultimas_melodias:
        plt.plot(mel, marker='o', linestyle='-', color='orange', alpha=0.1)

    # Draw final melody
    plt.plot(melodia_final, marker='o', linestyle='-', color='red', linewidth=4, label='Final melody')
    plt.xticks(fontsize=18)
    plt.yticks(
        range(len(escala_completa)),
        [escala_completa[i] if i % 12 == 0 else "" for i in range(len(escala_completa))],
        fontsize=18
    )
    plt.grid(True, axis='y', linestyle='--', alpha=0.5)
    plt.text(
        0.98, 0.02,
        f'Final melody:\n{" ".join(melodia_final_notas)}',
        fontsize=20, ha='right', va='bottom',
        transform=plt.gca().transAxes,
        bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray', boxstyle='round,pad=0.3')
    )
    plt.xlabel('Position in Melody', fontsize=22)
    plt.ylabel('Musical Note', fontsize=22)
    plt.legend(fontsize=17)
    plt.tight_layout()
    plt.savefig(f"melodia_T_{temperatura}.png", dpi=300)
    plt.show()

    # ---------------- ENERGY PLOT ----------------
    plt.figure(figsize=(12, 6))
    plt.plot(historico_energia, color='green')
    plt.title(f"Energy evolution at T = {temperatura}", fontsize=26)
    plt.xlabel('Simulation steps (x100)', fontsize=18)
    plt.ylabel('Total energy', fontsize=18)
    plt.grid(True)
    plt.savefig(f"energia_T_{temperatura}.png", dpi=300)
    plt.show()

    # ---------------- AUDIO EXPORT ----------------
    audio_melodia = render_melody(melodia_final_notas, per_note=0.5, gap=0.02, sr=SR)
    save_wav(f"melodia_gerada_T_{temperatura}.wav", audio_melodia, sr=SR)